In [1]:
from src.dataset.data_reader import BIDSDatasetReader
from src.analysis.registery import Registery
from src.utils.graphics import styled_print
from src.dataset.data_loader import DataLoader
from matplotlib import pyplot as plt
import config as config
import numpy as np
from pathlib import Path

In [2]:
def create_epochs_for_all(
        registery, trial_mode, trial_unit, experiment_mode, 
        trial_boundary, trial_type, modality, tmin, tmax
    ):
    styled_print('', 'Creating EPOCHS for all subjects', color='red')
    epochs = {}

    for (subject, session), raw in registery.raw_data.items():
        styled_print('', f'Loading for sub-{subject}, ses-{session}', color='green')
        data_loader = DataLoader(
            eeg_data=raw, trial_mode=trial_mode, trial_unit=trial_unit,
            experiment_mode=experiment_mode, trial_boundary=trial_boundary,
            trial_type=trial_type, modality=modality
        )
        try:
            epochs[(subject, session)] = data_loader.create_epochs(tmin=tmin, tmax=tmax)
        except Exception as e:
            styled_print('', f"Error processing sub-{subject}, ses-{session}: {e}", color='yellow')

    return epochs


In [3]:
registary = Registery()

╭──────────────────────────────╮
│  Initializing Register Class │
╰──────────────────────────────╯

Loading raw data for all subjects

Loading raw data for sub-01, ses-01

╭─────────────────────────────────────────╮
│ 🚀 Initializing BIDSDatasetReader Class │
╰─────────────────────────────────────────╯

Loading Raw Data

...

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.2s


Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 94.8 mm
Computing interpolation matrix from 59 sensor positions
Interpolating 3 sensors
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


Removing Artifacts

Fitting ICA to data using 62 channels (please be patient, this may take a while)
Omitting 399 of 6307000 (0.01%) samples, retaining 6306601 (99.99%) samples.
Selecting by number: 50 components


In [94]:

def plot_vs_fixation_all_subjects(
        registery, trial_mode, trial_unit, experiment_mode,
        trial_boundary, trial_type, modality, channels=None
    ):
    """Plots evoked responses in selected channels across all subjects."""

    epochs_pictorial = create_epochs_for_all(
        registery=registery,
        trial_mode=trial_mode, trial_unit=trial_unit,
        experiment_mode=experiment_mode, trial_boundary=trial_boundary,
        trial_type=trial_type, modality=modality,
        tmin=-0.2, tmax=0.5
    )
    print(epochs_pictorial)
    exit
    epochs_fixation = create_epochs_for_all(
        registery=registery,
        trial_mode=trial_mode, trial_unit=trial_unit,
        experiment_mode=experiment_mode, trial_boundary=trial_boundary,
        trial_type='Fixation', modality=modality,
        tmin=0.3, tmax=1.0
    )

    if not epochs_pictorial or not epochs_fixation:
        print("No epochs available for plotting.")
        return

    fig, axes = plt.subplots(5, 4, figsize=(15, 10), sharex=True, sharey=True)
    axes = axes.flatten()

    for i, ((subject, session), epoch) in enumerate(epochs_pictorial.items()):
        if (subject, session) not in epochs_fixation:
            print(f"Skipping subject {subject}, session {session}: No fixation data.")
            continue

        evoked_pictorial = epoch.average()
        evoked_fixation = epochs_fixation[(subject, session)].average()

        selected_channels = channels if channels else evoked_pictorial.ch_names
        if channels != None:
            evoked_pictorial.pick_channels(selected_channels)
            evoked_fixation.pick_channels(selected_channels)

        mean_signal_pictorial = np.mean(evoked_pictorial.data, axis=0)
        mean_signal_fixation = np.mean(evoked_fixation.data, axis=0)

        ax = axes[i]
        ax.plot(mean_signal_pictorial, label='Pictorial', color='green')
        ax.plot(mean_signal_fixation, label='Fixation', color='blue')

        ax.axvline(200, color='cyan', linestyle='--', label="Onset")
        ax.axvline(300, color='r', linestyle='--', label="100ms")
        ax.axvline(500, color='black', linestyle='--', label="300ms")

        ax.set_title(f"sub-{subject}_ses-{session}")
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        if i % 4 == 0:
            ax.set_ylabel("µV")
        if i >= 16:
            ax.set_xlabel("Time (s)")

    axes[0].legend(loc='upper right')
    plt.tight_layout()

    # Save figure
    images_dir = config.IMAGES_DIR
    image_filepath = Path(images_dir,picture_name)
    plt.savefig(image_filepath, dpi=600)
    print(f"Saved plot to {image_filepath}")


In [79]:
def create_epochs_for_all(
        registery, trial_mode, trial_unit, experiment_mode, 
        trial_boundary, trial_type, modality, tmin, tmax
    ):
    styled_print('', 'Creating EPOCHS for all subjects', color='red')
    epochs = {}

    for (subject, session), raw in registery.raw_data.items():
        styled_print('', f'Loading for sub-{subject}, ses-{session}', color='green')
        data_loader = DataLoader(
            eeg_data=raw, trial_mode=trial_mode, trial_unit=trial_unit,
            experiment_mode=experiment_mode, trial_boundary=trial_boundary,
            trial_type=trial_type, modality=modality
        )
        try:
            epochs[(subject, session)] = data_loader.create_epochs(tmin=tmin, tmax=tmax)
        except Exception as e:
            styled_print('', f"Error processing sub-{subject}, ses-{session}: {e}", color='yellow')

    return epochs


trial_mode = ''
trial_unit = 'Words'
experiment_mode = 'Experiment'
trial_boundary = 'Start'
trial_type = 'Stimulus'
modality = 'Pictures'
channels = ['PO3', 'POz', 'PO4']

epochs_pictorial = create_epochs_for_all(
        registery=registary,
        trial_mode=trial_mode, trial_unit=trial_unit,
        experiment_mode=experiment_mode, trial_boundary=trial_boundary,
        trial_type=trial_type, modality=modality,
        tmin=-0.2, tmax=0.5
)

Creating EPOCHS for all subjects

Loading for sub-01, ses-01

╭────────────────────────────────╮
│  Initializing DataLoader Class │
╰────────────────────────────────╯

Creating EPOCHS

        Extraction Criteria        
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Parameter          ┃ Value      ┃
┣━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ 🧪 Trial Mode      ┃            ┃
┃ 📏 Trial Unit      ┃ Words      ┃
┃ 🔗 Experiment Mode ┃ Experiment ┃
┃ 🎭 Trial Boundary  ┃ Start      ┃
┃ 🔬 Trial Type      ┃ Stimulus   ┃
┃ 📊 Modality        ┃ Pictures   ┃
┃ ⏱ Tmin             ┃ -0.2       ┃
┃ ⏳ Tmax            ┃ 0.5        ┃
┗━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━┛

...

Loading for sub-14, ses-01

╭────────────────────────────────╮
│  Initializing DataLoader Class │
╰────────────────────────────────╯

Creating EPOCHS

        Extraction Criteria        
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Parameter          ┃ Value      ┃
┣━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ 🧪 Trial Mode      ┃            ┃
┃ 📏 Trial Unit      ┃ Words      ┃
┃ 🔗 Experiment Mode ┃ Experiment ┃
┃ 🎭 Trial Boundary  ┃ Start      ┃
┃ 🔬 Trial Type      ┃ Stimulus   ┃
┃ 📊 Modality        ┃ Pictures   ┃
┃ ⏱ Tmin             ┃ -0.2       ┃
┃ ⏳ Tmax            ┃ 0.5        ┃
┗━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━┛

Not setting metadata
130 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 130 events and 701 original time points ...
0 bad epochs dropped


In [ ]:
plot_vs_fixation_all_subjects(
    registery=registary,
    trial_mode=trail_mode, 
    trial_unit=trail_unit, 
    experiment_mode=experiment_mode,
    trial_boundary=trail_boundary, 
    trial_type=trail_type, 
    modality=modality,
    #channels=channels
)

In [38]:
ep[('02','01')].ch_names

['Fp1',
 'Fz',
 'F3',
 'F7',
 'FT9',
 'FC5',
 'FC1',
 'C3',
 'T7',
 'EOG1',
 'CP5',
 'CP1',
 'Pz',
 'P3',
 'P7',
 'O1',
 'Oz',
 'O2',
 'P4',
 'P8',
 'EOG2',
 'CP6',
 'CP2',
 'Cz',
 'C4',
 'T8',
 'FT10',
 'FC6',
 'FC2',
 'F4',
 'F8',
 'Fp2',
 'AF7',
 'AF3',
 'AFz',
 'F1',
 'F5',
 'FT7',
 'FC3',
 'C1',
 'C5',
 'TP7',
 'CP3',
 'P1',
 'P5',
 'PO7',
 'PO3',
 'POz',
 'PO4',
 'PO8',
 'P6',
 'P2',
 'CPz',
 'CP4',
 'TP8',
 'C6',
 'C2',
 'FC4',
 'FT8',
 'F6',
 'AF8',
 'AF4',
 'F2',
 'FCz']